## Entrenamiento y Evaluación de Modelos BERT/BETO para Análisis de Sentimientos -variable 'Polarity' REST-MEX 2025

Este notebook documenta el proceso completo de entrenamiento, validación y evaluación de modelos de lenguaje basados en BERT/BETO para la clasificación de polaridad en textos del reto REST-MEX 2025. Incluye la preparación y limpieza de datos, tokenización, manejo de desbalanceo de clases, definición de métricas personalizadas, entrenamiento con GPU, generación de reportes de desempeño (matriz de confusión, métricas F1, precisión y recall por clase), así como la carga y evaluación de checkpoints previos. El flujo permite experimentar con diferentes configuraciones y facilita la interpretación de resultados para mejorar el desempeño del modelo.

### Lectura y limpieza de datos 

In [ ]:
import pandas as pd 
import os
import numpy as np 
import re
import unicodedata
import nltk
from nltk.corpus import stopwords

# Aseguramos que las stopwords estén disponibles
try:
    stopwords.words('spanish')
except LookupError:
    nltk.download('stopwords')

# Ruta de lectura
ruta = "../data"
archivo = os.path.join(ruta, "Rest-Mex_2025_train.csv") 

with open(archivo, 'r', encoding='utf-8', errors='replace') as f:
    Data = pd.read_csv(f)

# Arreglamos problemas de codificación
def arregla_mojibake(texto):
    try:
        return texto.encode('latin1').decode('utf-8')
    except:
        return texto

Data['Title'] = Data['Title'].fillna('').apply(arregla_mojibake)
Data['Review'] = Data['Review'].fillna('').apply(arregla_mojibake)

# Creamos columna base con texto leído y concatenado
Data['Texto_Leido'] = (Data['Title'] + ' ' + Data['Review']).str.strip()

# Función de limpieza profunda para modelos clásicos
stopwords_es = set(stopwords.words('spanish'))

def limpiar_texto(texto):
    texto = texto.lower()
    # Quitamos acentos
    texto = unicodedata.normalize('NFD', texto)
    texto = ''.join([char for char in texto if unicodedata.category(char) != 'Mn'])
    # Eliminamos caracteres no alfabéticos
    texto = re.sub(r'[^a-zñü\s]', '', texto)
    palabras = texto.split()
    palabras = [p for p in palabras if p not in stopwords_es]
    return ' '.join(palabras).strip()

# Generamos columna limpia
Data['Texto_Limpio'] = Data['Texto_Leido'].apply(limpiar_texto)

# Eliminamos columnas originales
Data = Data.drop(columns=['Title', 'Review'])

In [ ]:
# La celda anterior se puede resumir en:
import pandas as pd
import numpy as np
import os
# Ruta de lectura
ruta = r"C:\Users\uzgre\Codes\Python\Ciencia de Datos\Proyecto_final\Rest-Mex_2025_DataSet"
archivo = os.path.join(ruta, "Train_Limpio.csv") 

with open(archivo, 'r', encoding='utf-8', errors='replace') as f:
    Data = pd.read_csv(f)

### Verificamos CUDA

In [2]:
import torch
print(torch.cuda.is_available())  # True si está bien
print(torch.cuda.get_device_name(0))  # Nombre de tu GPU


True
NVIDIA GeForce RTX 3050 6GB Laptop GPU


### Clasificacion con BETO

In [4]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

# 1. Cargar tokenizer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

# 2. Asegurar que las etiquetas sean válidas (1 a 5) y convertirlas a 0-4
Data = Data[Data['Polarity'].isin([1.0, 2.0, 3.0, 4.0, 5.0])]
Data['label'] = Data['Polarity'].astype(int) - 1

# 3. Split de datos
train_texts, val_texts, train_labels, val_labels = train_test_split(
    Data['Texto_Leido'].tolist(),
    Data['label'].tolist(),
    test_size=0.2,
    stratify=Data['label']
)

# 4. Tokenización
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)



In [5]:

# 5. Dataset personalizado
class RMDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = RMDataset(train_encodings, train_labels)
val_dataset = RMDataset(val_encodings, val_labels)

# 6. Cálculo de pesos de clase (para el dataset desbalanceado)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
weights_tensor = torch.tensor(class_weights, dtype=torch.float)

# 7. Cargar modelo BERT con clasificación para 5 clases
model = AutoModelForSequenceClassification.from_pretrained(
    "dccuchile/bert-base-spanish-wwm-cased",
    num_labels=5
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 8. Definir función de métricas
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {"accuracy": acc, "f1": f1}


''''
from sklearn.metrics import accuracy_score, f1_score, 
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    # Accuracy y F1
    acc = accuracy_score(labels, preds)
    f1_macro = f1_score(labels, preds, average='macro')
    f1_micro = f1_score(labels, preds, average='micro')
    f1_weighted = f1_score(labels, preds, average='weighted')

    return {
        "accuracy": acc,
        "f1_macro": f1_macro,
        "f1_micro": f1_micro,
        "f1_weighted": f1_weighted,
    }

'''

# 9. Usar Trainer personalizado para incluir class_weights
from torch.nn import CrossEntropyLoss

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = CrossEntropyLoss(weight=weights_tensor.to(model.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


# 10. Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./resultados_beto",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True
)

# 11. Entrenador
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# 12. ENTRENAMIENTO en GPU (si está disponible)
trainer.train()

c:\Users\uzgre\anaconda3\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.857200,1.037642,0.640456,0.664765
2,0.902600,1.019604,0.735527,0.742114
3,0.733000,1.102260,0.714931,0.729511


TrainOutput(global_step=31209, training_loss=0.8307517970352004, metrics={'train_runtime': 11579.6044, 'train_samples_per_second': 43.121, 'train_steps_per_second': 2.695, 'total_flos': 3.284503772378112e+16, 'train_loss': 0.8307517970352004, 'epoch': 3.0})

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# 13. PREDICCIONES sobre el conjunto de validación
preds_output = trainer.predict(val_dataset)
logits = preds_output.predictions
preds = np.argmax(logits, axis=1)

# 14. MATRIZ DE CONFUSIÓN
cm = confusion_matrix(val_labels, preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[1,2,3,4,5], yticklabels=[1,2,3,4,5])
plt.xlabel("Etiqueta Predicha")
plt.ylabel("Etiqueta Verdadera")
plt.title("Matriz de Confusión BETO")
plt.tight_layout()
plt.show()

# 15. REPORTE DETALLADO
print("Reporte de Clasificación (F1, Precision, Recall por clase):")
print(classification_report(val_labels, preds, digits=3, target_names=["Muy Neg", "Neg", "Neutro", "Pos", "Muy Pos"]))


NameError: name 'val_labels' is not defined

### Reanudamos entrenamiento...

In [ ]:
!pip install datasets

In [3]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import torch
import os
from sklearn.metrics import accuracy_score

# 1. Preparación del Dataset
Data['Polarity'] = Data['Polarity'].astype(int)
Data = Data[Data['Polarity'].isin([1, 2, 3, 4, 5])]
Data['labels'] = Data['Polarity'] - 1  # Etiquetas 0 a 4
# Asegurar que todas las etiquetas estén entre 0 y 4
assert Data['labels'].between(0, 4).all(), f"Hay etiquetas fuera del rango 0-4: {Data['labels'].unique()}"

# Convertir a Dataset de Hugging Face
dataset = Dataset.from_pandas(Data[['Texto_Leido', 'labels']])

# 2. Tokenizador y modelo
model_path = "./resultados_beto/checkpoint-20806"  # ← tu checkpoint previo
tokenizer = BertTokenizerFast.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = BertForSequenceClassification.from_pretrained(model_path)

# 3. Tokenización
def tokenize(batch):
    return tokenizer(batch["Texto_Leido"], padding="max_length", truncation=True, max_length=128)

dataset = dataset.map(tokenize, batched=True)

# Asegurar que las etiquetas estén en formato torch.long
def cast_labels(batch):
    batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
    return batch

dataset = dataset.map(cast_labels)

dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

# 4. División en entrenamiento y validación
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# 5. Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./resultados_beto_mas_epocas",  # nuevo directorio
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,  # ← solo guarda el checkpoint más reciente
    num_train_epochs=6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs_mas_epocas",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True
)

# 6. Métricas de evaluación
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 8. Entrenamiento (¡a dormir!)
trainer.train(resume_from_checkpoint=model_path)



Map:   0%|          | 0/208051 [00:00<?, ? examples/s]

Map:   0%|          | 0/208051 [00:00<?, ? examples/s]

c:\Users\uzgre\anaconda3\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\uzgre\AppData\Local\Temp\ipykernel_12920\507870675.py:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\uzgre\anaconda3\Lib\site-packages\transformers\trainer.py:3423: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be exec

Epoch,Training Loss,Validation Loss,Accuracy
3,0.467000,0.517889,0.778640
4,0.445700,0.544375,0.780995
5,0.343200,0.614318,0.777631
6,0.260900,0.765110,0.771479


TrainOutput(global_step=62418, training_loss=0.25837980316252807, metrics={'train_runtime': 29535.1911, 'train_samples_per_second': 33.812, 'train_steps_per_second': 2.113, 'total_flos': 6.569007544756224e+16, 'train_loss': 0.25837980316252807, 'epoch': 6.0})

### Cargamos modelos ya entrenados y hacemos predicciones

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Seleccionar 3 textos al azar
muestras = Data[['Texto_Leido', 'Polarity']].sample(10, random_state=42).reset_index(drop=True)

# Cargar modelo y tokenizer
#checkpoint_path = "./resultados_beto_mas_epocas/checkpoint-41612"  # ← Ajusta con el número real
checkpoint_path = "./resultados_beto/checkpoint-20806"
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

# Enviar a dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Tokenizar entradas
inputs = tokenizer(
    list(muestras['Texto_Leido']),
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors="pt"
)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Predecir
with torch.no_grad():
    outputs = model(**inputs)
    preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()

# Mostrar resultados
for i, row in muestras.iterrows():
    print(f"\nTexto {i+1}: {row['Texto_Leido']}")
    print(f"→ Polaridad real: {row['Polarity']}  |  Predicción modelo: {preds[i]+1}")



Texto 1: MEXICANO Es un lugar bellísimo, para llegar es necesario transportarte en lancha la cual tiene un precio bastante accesible, en la isla venden varias artesanías del lugar, desde prendas de vestir, antojitos mexicano, recuerditos y demás, su gente es muy amable, la mayoria de las personas conservan su lengua nativa, el lugar es pequeño pero fácil de caminar en una tarde, guarda muchas tradiciones mexicanas que se han perdido y te transporta a un lugar perdido en el tiempo
→ Polaridad real: 4.0  |  Predicción modelo: 5

Texto 2: Una vista sensacional El lugar es muy agradable, con una vista realmente increíble, la atención de los meseros muy buena, pero la comida no es nada fuera de lo común e incluso las porciones son pequeñas. La carta de vinos pequeña, pero razonablemente completa.
→ Polaridad real: 3.0  |  Predicción modelo: 3

Texto 3: Bastante recomendable para pasar la tarde Es una plaza en forma de herradura con muchos juegos infantiles para niños, grandes jardines con 

### Resumen de metricas del checkpoint

In [3]:
import json

def mostrar_metricas_checkpoint(path):
    with open(f"{path}/trainer_state.json", "r", encoding="utf-8") as f:
        state = json.load(f)

    resumen = []
    for log in state.get("log_history", []):
        if "eval_loss" in log:
            resumen.append({
                "epoch": log.get("epoch"),
                "step": log.get("step"),
                "val_loss": round(log.get("eval_loss", 0), 6),
                "accuracy": round(log.get("eval_accuracy", 0), 6),
                "f1": round(log.get("eval_f1", 0), 6)  # ← solo si usaste F1 en compute_metrics
            })
    return resumen
resumen = mostrar_metricas_checkpoint("./resultados_beto/checkpoint-20806")
for r in resumen:
    print(r)


{'epoch': 1.0, 'step': 10403, 'val_loss': 1.037642, 'accuracy': 0.640456, 'f1': 0.664765}
{'epoch': 2.0, 'step': 20806, 'val_loss': 1.019604, 'accuracy': 0.735527, 'f1': 0.742114}


In [3]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer
from sklearn.metrics import accuracy_score, f1_score, classification_report
import torch
from datasets import Dataset
import pandas as pd

# 1. Preparación del Dataset
Data['Polarity'] = Data['Polarity'].astype(int)
Data = Data[Data['Polarity'].isin([1, 2, 3, 4, 5])]
Data['labels'] = Data['Polarity'] - 1  # Etiquetas 0 a 4

# Convertir a Dataset de Hugging Face
dataset = Dataset.from_pandas(Data[['Texto_Leido', 'labels']])

# 2. Tokenización
tokenizer = BertTokenizerFast.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

def tokenize(batch):
    return tokenizer(batch["Texto_Leido"], padding="max_length", truncation=True, max_length=128)

dataset = dataset.map(tokenize, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

# 3. Separar en evaluación y entrenamiento
dataset = dataset.train_test_split(test_size=0.2, seed=42)
eval_dataset = dataset["test"]

# 4. Cargar modelo desde el checkpoint
#model = BertForSequenceClassification.from_pretrained("./resultados_beto_mas_epocas/checkpoint-41612")
model = BertForSequenceClassification.from_pretrained("./resultados_beto/checkpoint-20806")

# 5. Crear Trainer solo para evaluar
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
)

# 6. Obtener predicciones
predictions = trainer.predict(eval_dataset)
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids

# 7. Calcular métricas
acc = accuracy_score(labels, preds)
f1_macro = f1_score(labels, preds, average="macro")
f1_micro = f1_score(labels, preds, average="micro")
f1_weighted = f1_score(labels, preds, average="weighted")

print(f"Accuracy: {acc:.4f}")
print(f"F1 Macro: {f1_macro:.4f}")
print(f"F1 Micro: {f1_micro:.4f}")
print(f"F1 Weighted: {f1_weighted:.4f}")
print("\nReporte completo:\n")
print(classification_report(labels, preds, digits=4))


Map:   0%|          | 0/208051 [00:00<?, ? examples/s]

C:\Users\uzgre\AppData\Local\Temp\ipykernel_4952\2242123223.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Accuracy: 0.7715
F1 Macro: 0.6953
F1 Micro: 0.7715
F1 Weighted: 0.7780

Reporte completo:

              precision    recall  f1-score   support

           0     0.7195    0.8654    0.7857      1070
           1     0.6489    0.5452    0.5926      1139
           2     0.7182    0.5850    0.6448      3072
           3     0.5257    0.6743    0.5908      9073
           4     0.8975    0.8306    0.8628     27257

    accuracy                         0.7715     41611
   macro avg     0.7020    0.7001    0.6953     41611
weighted avg     0.7918    0.7715    0.7780     41611

